In [ ]:
ENV_TYPE = "TEST1"

if(ENV_TYPE != "TEST"):
  !git clone "https://github.com/kk-digital/kcg-ml-sd1p4.git"
  %cd kcg-ml-sd1p4
  !pip3 install -r requirements.txt
  exit()
else:
  !pip3 install -r requirements.txt --ignore-installed

Cloning into 'kcg-ml-sd1p4'...
remote: Enumerating objects: 5363, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 5363 (delta 16), reused 18 (delta 7), pack-reused 5331
Receiving objects: 100% (5363/5363), 104.85 MiB | 13.63 MiB/s, done.
Resolving deltas: 100% (3417/3417), done.
/content/kcg-ml-sd1p4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.9/934.9 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import os, sys
if(os.getcwd() == "/content"):
  %cd kcg-ml-sd1p4
import torch
from typing import List
import hashlib
import json
import math
import numpy as np
import safetensors as st
from os.path import join

from chad_score.chad_score import ChadScorePredictor
from stable_diffusion import StableDiffusion
from stable_diffusion.model.clip_text_embedder import CLIPTextEmbedder
from stable_diffusion.model.clip_image_encoder import CLIPImageEncoder
from stable_diffusion.utils_model import *
from stable_diffusion.utils_backend import *
from stable_diffusion.utils_model import initialize_latent_diffusion
from stable_diffusion.utils_image import *
from stable_diffusion.constants import IODirectoryTree
from stable_diffusion.constants import TOKENIZER_PATH, TEXT_MODEL_PATH
from transformers import CLIPTextModel, CLIPTokenizer


base_dir = "./"
sys.path.insert(0, base_dir)

# Variables
SEED = 0
BATCH_SIZE = 1
POPULATION_SIZE = 12
GEN_IMAGE_N_GENERATIONS = 50
CFG_STRENGTH = 9
N_STEPS = 20
EMBEDDED_PROMPTS_DIR = os.path.abspath(join(base_dir, "./input/embedded_prompts/"))
OUTPUT_DIR = os.path.abspath(
    os.path.join(base_dir, "./output/ga/")
)
IMAGES_DIR = os.path.abspath(join(OUTPUT_DIR, "images/"))
FEATURES_DIR = os.path.abspath(join(OUTPUT_DIR, "features/"))

NULL_PROMPT = ""
# DEVICE = input("Set device: 'cuda:i' or 'cpu'")
DEVICE = get_device()

print(EMBEDDED_PROMPTS_DIR)
print(OUTPUT_DIR)
print(IMAGES_DIR)
print(FEATURES_DIR)

os.makedirs(EMBEDDED_PROMPTS_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(IMAGES_DIR, exist_ok=True)
os.makedirs(FEATURES_DIR, exist_ok=True)

/content/kcg-ml-sd1p4


INFO:root:Using CUDA device <built-in method index of str object at 0x7cc78dd3a330>: Tesla T4
INFO: Using CUDA device <built-in method index of str object at 0x7cc78dd3a330>: Tesla T4


/content/kcg-ml-sd1p4/input/embedded_prompts
/content/kcg-ml-sd1p4/output/ga
/content/kcg-ml-sd1p4/output/ga/images
/content/kcg-ml-sd1p4/output/ga/features


In [ ]:
!git checkout genetic-algorithm
!git pull

In [ ]:
!python3 ./setup_directories_and_models.py --download_base_clip_model True --download_base_sd_model True

In [4]:
import random

# Function to generate prompts
def generate_prompts(prompt_segments, num_prompts=6):
    # Select 6 random segments from the prompt_segments list
    selected_prompts = random.sample(prompt_segments, num_prompts)
    modifiers = [
        'beautiful', 'gorgeous', 'stunning', 'charming', 'captivating', 'breathtaking',
        'masterpiece', 'exquisite', 'magnificent', 'majestic', 'elegant', 'sublime',
        'ugly', 'hideous', 'grotesque', 'repulsive', 'disgusting', 'revolting',
        'futuristic', 'cyberpunk', 'hi-tech', 'advanced', 'innovative', 'modern',
        'fantasy', 'mythical', 'scifi', 'side scrolling', 'character', 'side scrolling',
        'white background', 'centered', 'full character', 'no background', 'not centered',
        'line drawing', 'sketch', 'black and white', 'colored', 'offset', 'video game']

    # Add modifiers to the selected prompts and return them in a list (PROMPT)
    return [f"{random.choice(modifiers)} {prompt}" for prompt in selected_prompts]

# List of prompt segments
prompt_segments = ['chibi', 'waifu', 'cyborg', 'dragon', 'android', 'nekomimi', 'mecha', 'kitsune', 'AI companion', 'furry detective', 'robot butler', 'futuristic steampunk', 'cybernetic implants',
                   'anthropomorphic AI', 'mechanical wizard', 'kemonomimi', 'android rebellion', 'magical robot pet', 'intergalactic furball', 'cyberpunk android', 'shapeshifting furry',
                   'mech pilot', 'furry time traveler']

# Generate 6 random prompts with modifiers (initial population)
PROMPT = generate_prompts(prompt_segments)

# Print the generated prompts
print(PROMPT)

['captivating kitsune', 'majestic intergalactic furball', 'character anthropomorphic AI', 'beautiful robot butler', 'disgusting android rebellion', 'futuristic mecha']


In [6]:
def embed_and_save_prompts(prompts: list):
    null_prompt = ""
    clip_text_embedder = CLIPTextEmbedder(device=get_device())
    clip_text_embedder.load_submodels()

    null_cond = clip_text_embedder(null_prompt)

    # Initialize list to store embeds
    embedded_prompts_list = []

    for prompt in prompts:
        # Embed each prompt individually
        print("Generating prompt ", prompt)
        embedded_prompt = clip_text_embedder(prompt)
        embedded_prompts_list.append(embedded_prompt)

    clip_text_embedder.to("cpu")
    del clip_text_embedder
    torch.cuda.empty_cache()
    get_memory_status()

    return (embedded_prompts_list, null_cond)

def normalized(a, axis=-1, order=2):
    import numpy as np

    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2 == 0] = 1
    return a / np.expand_dims(l2, axis)

# Generate embeddings for each prompt
embedding, null_prompt = embed_and_save_prompts(PROMPT)
num_images = len(embedding)

INFO:root:Using CUDA device <built-in method index of str object at 0x7cc78dd3a330>: Tesla T4
INFO: Using CUDA device <built-in method index of str object at 0x7cc78dd3a330>: Tesla T4
INFO:root:Using CUDA device <built-in method index of str object at 0x7cc78dd3a330>: Tesla T4
INFO: Using CUDA device <built-in method index of str object at 0x7cc78dd3a330>: Tesla T4


Tokenizer successfully loaded from : ./input/model/clip/text_embedder/tokenizer

CLIP text model successfully loaded from : ./input/model/clip/text_embedder/text_model

Generating prompt  captivating kitsune
Generating prompt  majestic intergalactic furball
Generating prompt  character anthropomorphic AI
Generating prompt  beautiful robot butler
Generating prompt  disgusting android rebellion
Generating prompt  futuristic mecha


INFO:root:Using device: None
INFO: Using device: None


In [ ]:
# Load Stable Diffusion
sd = StableDiffusion(device=DEVICE, n_steps=N_STEPS)
sd.quick_initialize()
sd.model.load_unet()
sd.model.load_autoencoder().load_decoder()


# Load chadscore and clip
import clip

# Test calculate chadscore
chad = ChadScorePredictor(768)
chad.load_model("./input/model/aesthetic_scorer/sac+logos+ava1-l14-linearMSE.pth")
image_features_clip_model, preprocess = clip.load("ViT-L/14", device=get_device())  #RN50x64

In [19]:
def generate_images_from_embeddings(embedded_prompts_array, null_prompt):
  # 'embedded_prompts_array' is the array of shape (i, 77, 768)
  image = sd.generate_images_from_embeddings(
      seed=SEED, embedded_prompt=embedded_prompts_array[i:i+1], null_prompt=null_prompt
  )
  return image


solution_reshaped = best_solution.reshape(1, 77, 768)
solution_reshaped = torch.tensor(solution_reshaped, dtype=torch.float32)

image = generate_images_from_embeddings(solution_reshaped, null_prompt)
    pil_image = to_pil(image[0])
    filename=f"{IMAGES_DIR}/{generation}.png"
    pil_image.show()
    pil_image.save(filename)

In [38]:
import pygad

# Function to calculate the chad score for batch of images
def calculate_chad_score(ga_instance, solution, solution_idx):
  # Convert the solution back to the original shape (1, 77, 768)
  solution_reshaped = solution.reshape(1, 77, 768)

  # Convert the numpy array to a PyTorch tensor
  solution_reshaped = torch.tensor(solution_reshaped, dtype=torch.float32)

  # Copy the tensor to CUDA device if 'device' is 'cuda'
  if device == 'cuda':
    solution_reshaped = solution_reshaped.to(device)

  # Generate an image using the solution
  image = generate_images_from_embeddings(solution_reshaped, null_prompt)

  pil_image = to_pil(image[0])  # Convert to (height, width, channels)

  # If generation % 1 == 0 save
  if(ga_instance.generations_complete % 2 == 0):
    os.makedirs(f"{IMAGES_DIR}/{ga_instance.generations_complete}", exist_ok=True)
    filename=f"{IMAGES_DIR}/{ga_instance.generations_complete}/{idx}.png"
    pil_image.show()
    pil_image.save(filename)

  unsqueezed_image = preprocess(pil_image).unsqueeze(0).to(device)

  with torch.no_grad():
    image_features = image_features_clip_model.encode_image(unsqueezed_image)

    im_emb_arr = normalized(image_features.cpu().detach().numpy() )
    prediction = chad(torch.from_numpy(im_emb_arr).to(device).type(torch.cuda.FloatTensor))
    chad_score = prediction.item()
  return chad_score

# Define the GA loop function
def genetic_algorithm_loop(sd, embedded_prompts, null_prompt, generations=10, population_size=POPULATION_SIZE, mutation_rate=0.4, num_parents_mating=2):

    # Convert each tensor in the list to a 2D numpy array and store them in a new list
    embedded_prompts_list = []
    num_genes = embedded_prompts[0].shape[0] * embedded_prompts[0].shape[1]

    for tensor in embedded_prompts:
        tensor = tensor.cpu()
        list_tensor = tensor.squeeze().tolist()
        embedded_prompts_list.append(list_tensor)

    # Turn into a numpy array
    embedded_prompts_list = np.hstack(embedded_prompts_list)

    # Initialize the GA
    ga_instance = pygad.GA(num_generations=generations,
                              num_parents_mating=num_parents_mating,
                              fitness_func=calculate_chad_score,
                              num_genes=num_genes,
                              crossover_type="single_point",
                              initial_population=embedded_prompts_list,
                              mutation_percent_genes=mutation_rate*100)


    # Get the final best solution and images
    best_solution, best_solution_fitness = ga_instance.best_solution()
    best_solution = best_solution.reshape(1, 77, 768)  # Reshape the best solution to the correct shape
    final_image = sd.generate_images_from_embeddings(embedded_prompt=best_solution, null_prompt=null_prompt)

    # Save the final best solution images
    final_filename=f"{IMAGES_DIR}/final.png"
    pil_final = to_pil(final_image[0])
    pil_final.show()
    pil_final.save(final_filename)

    return best_solution

# Call the GA loop function with your initialized StableDiffusion model
best_solution = genetic_algorithm_loop(sd, embedding, null_prompt)

print("Best solution found!")

torch.save(embedded_prompts, join(EMBEDDED_PROMPTS_DIR, "embedded_final_solution.pt"))
print("Saving solution...")

cannot reshape array of size 4608 into shape (1,77,768)
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pygad/pygad.py", line 1669, in cal_pop_fitness
    fitness = self.fitness_func(self, sol, sol_idx)
  File "<ipython-input-38-18cf46e97513>", line 6, in calculate_chad_score
    solution_reshaped = solution.reshape(1, 77, 768)
ValueError: cannot reshape array of size 4608 into shape (1,77,768)
ERROR:pygad.pygad:cannot reshape array of size 4608 into shape (1,77,768)
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pygad/pygad.py", line 1669, in cal_pop_fitness
    fitness = self.fitness_func(self, sol, sol_idx)
  File "<ipython-input-38-18cf46e97513>", line 6, in calculate_chad_score
    solution_reshaped = solution.reshape(1, 77, 768)
ValueError: cannot reshape array of size 4608 into shape (1,77,768)
ERROR: cannot reshape array of size 4608 into shape (1,77,768)
Traceback (most recent call last):
  File "/usr/local

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pygad/pygad.py", line 1669, in cal_pop_fitness
    fitness = self.fitness_func(self, sol, sol_idx)
  File "<ipython-input-38-18cf46e97513>", line 6, in calculate_chad_score
    solution_reshaped = solution.reshape(1, 77, 768)
ValueError: cannot reshape array of size 4608 into shape (1,77,768)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-38-18cf46e97513>", line 76, in <cell line: 76>
    best_solution = genetic_algorithm_loop(sd, embedding, null_prompt)
  File "<ipython-input-38-18cf46e97513>", line 63, in genetic_algorithm_loop
    best_solution, best_solution_fitness = ga_instance.best_solution()
  File "/usr/local/lib/python3.10/dist-packages/pygad/pygad.py", line

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/pygad/pygad.py:1669 in cal_pop_fitness                   │
│                                                                                                  │
│   1666 │   │   │   │   │   else:                                                                 │
│   1667 │   │   │   │   │   │   # Check if batch processing is used. If not, then calculate this  │
│   1668 │   │   │   │   │   │   if self.fitness_batch_size in [1, None]:                          │
│ ❱ 1669 │   │   │   │   │   │   │   fitness = self.fitness_func(self, sol, sol_idx)               │
│   1670 │   │   │   │   │   │   │   if type(fitness) in GA.supported_int_float_types:             │
│   1671 │   │   │   │   │   │   │   │   pass                                                      │
│   1672 │   │   │   │   │   │   │   else:                                                         │
│ in calculate_chad_score:6                                                                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: cannot reshape array of size 4608 into shape (1,77,768)

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553 in run_code        │
│                                                                                                  │
│   3550 │   │   │   │   elif async_ :                                                             │
│   3551 │   │   │   │   │   await eval(code_obj, self.user_global_ns, self.user_ns)               │
│   3552 │   │   │   │   else:                                                                     │
│ ❱ 3553 │   │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                     │
│   3554 │   │   │   finally:                                                                      │
│   3555 │   │   │   │   # Reset our crash handler in place                                        │
│   3556 │   │   │   │   sys.excepthook = old_excepthook                                           │
│ in <cell line: 76>:76                                                                            │
│ in genetic_algorithm_loop:63                                                                     │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/pygad/pygad.py:2154 in best_solution                     │
│                                                                                                  │
│   2151 │   │   try:                                                                              │
│   2152 │   │   │   if pop_fitness is None:                                                       │
│   2153 │   │   │   │   # If the 'pop_fitness' parameter is not passed, then we have to call the  │
│ ❱ 2154 │   │   │   │   pop_fitness = self.cal_pop_fitness()                                      │
│   2155 │   │   │   # Verify the type of the 'pop_fitness' parameter.                             │
│   2156 │   │   │   elif type(pop_fitness) in [tuple, list, numpy.ndarray]:                       │
│   2157 │   │   │   │   # Verify that the length of the passed population fitness matches the le  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/pygad/pygad.py:1805 in cal_pop_fitness                   │
│                                                                                                  │
│   1802 │   │   │   pop_fitness = numpy.array(pop_fitness)                                        │
│   1803 │   │   except 

In [ ]:
# Clean unused loaded models
del image_features_clip_model, sd